# Programming Assignment 2

Welcome to the second programming assignment for CS 4650! In this project, we will train LSTM POS-taggers, which take in a sentence and outputs part-of-speech labels for every word in the sentence.

We will use English text from the Wall Street Journal, marked with POS tags such as `NNP` (proper noun) and `DT` (determiner).

(Instructor: Wei Xu; TAs: Marcus Ma, Mounica Maddela, Ben Podrazhansky, Rahul Katre)

**To begin this project, make a copy of this notebook and save it to your local drive so that you can edit it.**


If you want GPU's (which will improve training speed), you can always change your instance type to GPU by going to Runtime -> Change runtime type -> Hardware accelerator.

If you're new to PyTorch, or simply want a refresher, we recommend you start by looking through these [Introduction to PyTorch](https://cocoxu.github.io/CS4650_spring2023/slides/PyTorch_tutorial.pdf) slides and this interactive [PyTorch Basics notebook](http://bit.ly/pytorchbasics). Additionally, this [Text Sentiment](http://bit.ly/pytorchexample) notebook will provide some insight into working with PyTorch for NLP specific problems. 


## Part 0 Colab Setup [DO NOT MODIFY]

Below, we will import our libraries and check for GPU usage.

In [1]:
# DO NOT MODIFY #
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# this is how we select a GPU if it's avalible on your computer or in the Colab environment.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

You can check to make sure a GPU is available using the following code block.

If the below message is shown, it means you are using a CPU.
```
/bin/bash: nvidia-smi: command not found
```

---





In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Mar  8 16:13:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!curl https://raw.githubusercontent.com/cocoxu/CS4650_projects_spring2023/master/p2_train.txt > train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2775k  100 2775k    0     0  7521k      0 --:--:-- --:--:-- --:--:-- 7521k


## Part 1 Data Preparation [10 points]

### Part 1.1 Loading Data [DO NOT MODIFY]

`train.txt`: The training data is present in this file. This file contains sequences of words and their respective tags. The data is split into 80% training and 20% development to train the model and tune the hyperparameters, respectively. See `load_tag_data` for details on how to read the training data.

In [4]:
# DO NOT MODIFY

def load_tag_data(tag_file):
    all_sentences = []
    all_tags = []
    sent = []
    tags = []
    with open(tag_file, 'r') as f:
        for line in f:
            if line.strip() == "":
                all_sentences.append(sent)
                all_tags.append(tags)
                sent = []
                tags = []
            else:
                word, tag, _ = line.strip().split()
                sent.append(word)
                tags.append(tag)
    return all_sentences, all_tags

train_sentences, train_tags = load_tag_data('train.txt')

unique_tags = set([tag for tag_seq in train_tags for tag in tag_seq])

# Create train-val split from train data
train_val_data = list(zip(train_sentences, train_tags))
random.shuffle(train_val_data)

print("Data Length: ", len(train_val_data))
print("Total tags: ", len(unique_tags))

Data Length:  8935
Total tags:  44


### Part 1.2 Training-Validation Splits

We need to split the data into training and validation splits. We will not be using a test split for this project. Implement `train_validation_split` in the cell below. 

In [5]:
# train_validation_split
# This method takes in a list of features and labels and splits them into train/val splits.
# Note how we are not creating a test set for this project.
#
# args:
# data - list of the tuple (sentence, tags)
# labels - list of POS tags for each corresponding sentence
# split - split proportion for training and validation
#
# returns:
# train_split, test_split
def train_validation_split(data, split=0.8):
    train_split, test_split = None, None
    #############################################################################
    # TODO: Implement the train-validation split
    # Hint: Referencing Project 1 for this function and the subsequent functions
    # could prove useful.
    #############################################################################
    N = len(data)
    I = int(N*split)
    train_split = data[:I]
    test_split = data[I:]
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    return train_split, test_split


Testing our function:

In [6]:
# testing train_validation_split
training_data, val_data = train_validation_split(train_val_data)
print(f'Training data proportion: {len(training_data) / len(train_val_data)}')
print(f'Validation data proportion: {len(val_data) / len(train_val_data)}')

Training data proportion: 0.8
Validation data proportion: 0.2


### Part 1.3 Word-to-Index and Tag-to-Index mapping
In order to work with text in Tensor format, we need to map each word and each tag to a unique index. Implement `create_word_and_tag_dicts` in the cell below.

In [7]:
# create_word_and_tag_dicts
# This method takes a collection of sentences and tags and produces three separate
# dictionaries that will be used later on.
# 
# args:
# data - tuple of (sentences, tags) that we will use to build our dictionary.
#
# returns:
# word_to_idx - dict[str] -> int
#        dictionary that maps all of the words in the vocabulary to a unique integer
#        representation.
#
# tag_to_idx - dict[str] -> int
#       dictionary that maps each tag to a unique integer representation.
#
# idx_to_tag - dict[int] -> str
#       dictionary that maps each integer from tag_to_idx to its original tag.
#       essentially, the inverse of tag_to_idx.
def create_word_and_tag_dicts(sentences, unique_tags):
    word_to_idx, tag_to_idx, idx_to_tag = {}, {}, {}
    #############################################################################
    # TODO: Implement create_word_and_tag_dicts
    #############################################################################
    
    # update word_to_idx
    w = 0
    
    for sentence in sentences:
      for word in sentence:
        if word not in word_to_idx:
          word_to_idx[word] = w
          w += 1

    word_to_idx[' '] = len(word_to_idx)
    # update tag_to_idx
    t = 0
    for tag in unique_tags:
      if tag not in tag_to_idx:
        tag_to_idx[tag] = t
        t += 1

    # update idx_to_tag
    idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    return word_to_idx, tag_to_idx, idx_to_tag

Testing our function:

In [8]:
word_to_idx, tag_to_idx, idx_to_tag = create_word_and_tag_dicts(train_sentences, unique_tags)

print("Total tags", len(tag_to_idx))
print("Vocab size", len(word_to_idx))

Total tags 44
Vocab size 19122


### Part 1.4 Prepare Sequence
Now we'll put everything together! `prepare_sequence` takes in a sentence and its corresponding tags, and returns the data transformed into index Tensors to be used for training in our model.

In [9]:
# prepare_sequence
# This method takes a sentence-tag pair and returns two Long-Tensors of the indices
# to be used for the LSTM model.
#
# returns:
# sentence_tensor - torch.LongTensor where each element in the tensor corresponds to
# the index of the word in the sentence
# tag_tensor - torch.LongTensor where each element in the tensor corresponds to
# the index of the tag
def prepare_sequence(sentence, tags, word_to_idx, tag_to_idx):
    sentence_tensor = torch.empty(len(sentence), dtype=torch.long)
    tag_tensor = torch.empty(len(tags), dtype=torch.long)
    #############################################################################
    # TODO: Implement prepare_sequence
    #############################################################################
    
    # update sentence_tensor
    for idx, word in enumerate(sentence):
      sentence_tensor[idx] = word_to_idx[word]
      
    # update tag_tensor
    for idx, tag in enumerate(tags):
      tag_tensor[idx] = tag_to_idx[tag]
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################
    return sentence_tensor, tag_tensor

## Part 2 Word-Level POS Tagger [20 points]
### Part 2.1 Set up model
We will build and train a Basic POS Tagger which is an LSTM model to tag the parts of speech in a given sentence using word-level information.


First we need to define some default hyperparameters.

In [10]:
EMBEDDING_DIM = 4
HIDDEN_DIM = 8
LEARNING_RATE = 0.1
LSTM_LAYERS = 1
DROPOUT = 0
EPOCHS = 10

### Part 2.2 Define Model

The model takes as input a sentence as a tensor in the index space. This sentence is then converted to embedding space where each word maps to its word embedding. The word embeddings is learned as part of the model training process. These word embeddings act as input to the LSTM which produces a representation for each word. Then the representations of words are passed to a Linear layer.

In [11]:
from torch.nn.utils.rnn import pad_packed_sequence
class BasicPOSTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(BasicPOSTagger, self).__init__()
        #############################################################################
        # TODO: Define and initialize anything needed for the forward pass.
        # You are required to create a model with:
        # an embedding layer: that maps words to the embedding space
        # an LSTM layer: that takes word embeddings as input and outputs hidden states
        # a linear layer: maps from hidden state space to tag space
        #############################################################################
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers = LSTM_LAYERS, bidirectional=True)
        self.linear = nn.Linear(hidden_dim*2, tagset_size)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def forward(self, sentence):
        tag_scores = None
        #############################################################################
        # TODO: Implement the forward pass.
        # Given a tokenized index-mapped sentence as the argument, 
        # compute the corresponding raw scores for tags (without softmax)
        # returns:: tag_scores (Tensor)
        #############################################################################
        emb = self.embedding(sentence)
        self.lstm.flatten_parameters()
        lstm_out, (hidden, c) = self.lstm(emb.view(len(sentence), 1, -1))
        tag_scores = self.linear(lstm_out.view(len(sentence), -1))
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        return tag_scores

### Part 2.3 Training

We define train and evaluate procedures that allow us to train our model using our created train-val split.

In [26]:
def train(epoch, model, loss_function, optimizer):
    model.train()
    train_loss = 0
    train_examples = 0
    for sentence, tags in training_data:
        #############################################################################
        # TODO: Implement the training method
        # Hint: you can use the prepare_sequence method for creating index mappings 
        # for sentences. Find the gradient with respect to the loss and update the
        # model parameters using the optimizer.
        #############################################################################
        #zero out the parameter gradients
        optimizer.zero_grad()

        #prepare input data (sentences and gold labels)
        prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
        prep_sentence = prep_sentence.to(device)
        prep_tags = prep_tags.to(device)

        #do forward pass with current input
        output = model(prep_sentence)
        
        #get loss with model predictions and true labels
        loss = loss_function(output, prep_tags)
        
    
        #update model parameters
        loss.backward()
        optimizer.step()

        #increase running total loss and the number of past training samples 
        train_loss += loss.item()
        train_examples += len(prep_tags)
        
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
    avg_train_loss = train_loss / train_examples
    avg_val_loss, val_accuracy = evaluate(model, loss_function)
        
    print("Epoch: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\t Val Accuracy: {:.0f}".format(epoch, 
                                                                      EPOCHS, 
                                                                      avg_train_loss, 
                                                                      avg_val_loss,
                                                                      val_accuracy))

def evaluate(model, loss_function):
  # returns:: avg_val_loss (float)
  # returns:: val_accuracy (float)
    model.eval()
    correct = 0
    val_loss = 0
    val_examples = 0
    with torch.no_grad():
        for sentence, tags in val_data:
            #############################################################################
            # TODO: Implement the evaluate method
            # Find the average validation loss along with the validation accuracy.
            # Hint: To find the accuracy, argmax of tag predictions can be used.
            #############################################################################
            
            #prepare input data (sentences and gold labels)
            prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
            prep_sentence = prep_sentence.to(device)
            prep_tags = prep_tags.to(device)

            #do forward pass with current batch of input
            output = model(prep_sentence)
            
            #prep_tags_reshape = prep_tags.view(1, -1)
            #output_reshaped = output.view(-1, len(tag_to_idx))
            #get loss with model predictions and true labels
            loss = loss_function(output, prep_tags)
            
            #get the predicted labels
            predictions = torch.argmax(output, dim=1)
            
            #get number of correct prediction
            correct += (predictions == prep_tags).sum().item()
            #increase running total loss and the number of past valid samples
            val_loss += loss.item()
            val_examples += len(prep_tags)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
    val_accuracy = 100. * correct / val_examples
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, val_accuracy

In [31]:
#############################################################################
# TODO: Initialize the model, optimizer and the loss function
#############################################################################

model = BasicPOSTagger(vocab_size         = len(word_to_idx.keys()),
                            embedding_dim = EMBEDDING_DIM,
                            hidden_dim    = HIDDEN_DIM,
                            tagset_size=len(tag_to_idx.keys())).to(device)

loss_function = torch.nn.CrossEntropyLoss(reduction='sum').to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    train(epoch, model, loss_function, optimizer)

Epoch: 1/10	Avg Train Loss: 1.0072	Avg Val Loss: 0.8537	 Val Accuracy: 76
Epoch: 2/10	Avg Train Loss: 0.7778	Avg Val Loss: 0.8329	 Val Accuracy: 78
Epoch: 3/10	Avg Train Loss: 0.6939	Avg Val Loss: 0.7715	 Val Accuracy: 78
Epoch: 4/10	Avg Train Loss: 0.6938	Avg Val Loss: 0.7766	 Val Accuracy: 78
Epoch: 5/10	Avg Train Loss: 0.6892	Avg Val Loss: 0.8474	 Val Accuracy: 78
Epoch: 6/10	Avg Train Loss: 0.6746	Avg Val Loss: 0.7138	 Val Accuracy: 80
Epoch: 7/10	Avg Train Loss: 0.6674	Avg Val Loss: 0.7303	 Val Accuracy: 80
Epoch: 8/10	Avg Train Loss: 0.6603	Avg Val Loss: 0.7634	 Val Accuracy: 78
Epoch: 9/10	Avg Train Loss: 0.6649	Avg Val Loss: 0.7845	 Val Accuracy: 79
Epoch: 10/10	Avg Train Loss: 0.6444	Avg Val Loss: 0.7252	 Val Accuracy: 80


**Sanity Check!** Under the default hyperparameter setting, after 5 epochs you should be able to get at least 75% accuracy on the validation set.

### Part 2.4 Error analysis

In this step, we will analyze what kind of errors it was making on the validation set.

Step 1, write a method to generate predictions from the validation set. For every sentence, get its words, predicted tags (model_tags), and the ground truth tags (gt_tags). To make the next step easier, you may want to concatenate words from all sentences into a very long list, and same for model_tags and gt_tags.


Step 2, analyze what kind of errors the model was making. For example, it may frequently label NN as VB. Let's get the top-10 most frequent types of errors, each of their frequency, and some example words. One example is at below. It is interpreted as the model predicts NNP as VBG for 626 times, with five random example words of this error being shown.

```
['VBG', 'NNP', 626, ['Rowe', 'Livermore', 'Parker', 'F-16', 'HEYNOW']]
```

In [32]:
#############################################################################
# TODO: Generate predictions for val_data
# Create lists of words, tags predicted by the model and ground truth tags.
# Hint: It should look very similar to the evaluate function.
#############################################################################
def generate_predictions(model, val_data):
    # returns:: word_list (str list)
    # returns:: model_tags (str list)
    # returns:: gt_tags (str list)
    # Your code here

    word_list = []
    gt_tags = []
    model_tags = []
    with torch.no_grad():
        for sentence, tags in val_data:

            prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
            prep_sentence = prep_sentence.to(device)
            prep_tags = prep_tags.to(device)
            output = model(prep_sentence)

            predictions = torch.argmax(output, dim=1)

            for word, tag, pred_tag in zip(sentence, prep_tags, predictions):
              word_list.append(word)
              gt_tags.append(idx_to_tag[tag.item()])
              model_tags.append(idx_to_tag[pred_tag.item()])

    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return word_list, model_tags, gt_tags

#############################################################################
# TODO: Carry out error analysis
# From those lists collected from the above method, find the 
# top-10 tuples of (model_tag, ground_truth_tag, frequency, example words)
# sorted by frequency
#############################################################################
def error_analysis(word_list, model_tags, gt_tags):
    # returns: errors (list of tuples)
    # Your code here
    frequencies = {}
    example_words = {}
    for word, model_tag, gt_tag in zip(word_list, model_tags, gt_tags):
      if model_tag != gt_tag: 
        if (model_tag, gt_tag) in frequencies:
          frequencies[(model_tag, gt_tag)] += 1
          example_words[(model_tag, gt_tag)].append(word)
        else:
          frequencies[(model_tag, gt_tag)] = 1
          example_words[(model_tag, gt_tag)] = [word]
    
    errors = []
    sorted_frequencies = dict(sorted(frequencies.items(), key=lambda x: x[1], reverse=True))
    for error in sorted_frequencies:
      e = (error[0], error[1], frequencies[error], example_words[error][:5])
      errors.append(e)
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return errors

word_list, model_tags, gt_tags = generate_predictions(model, val_data)
errors = error_analysis(word_list, model_tags, gt_tags)

for i in errors[:10]:
  print(i)

('NNP', 'NN', 623, ['yacht', 'hotdog', 'humor', 'stockpile', 'depressant'])
('NNP', 'NNS', 589, ['buddies', 'receipts', 'ounces', 'pleas', 'cities'])
('NNP', 'VBN', 580, ['created', 'been', 'priced', 'stopped', 'adjusted'])
('NNP', 'VBG', 541, ['declining', 'buying', 'laughing', 'contemplating', 'consuming'])
('NNP', 'JJ', 487, ['slick-talking', 'snake-oil', 'gullible', 'Callable', 'ever-narrowing'])
('NN', 'JJ', 361, ['greedy', 'uncomfortable', 'own', 'one-year', 'net'])
('NNP', 'CD', 308, ['1999', '1994', '101.60', '9.07', '396,000'])
('NN', 'VB', 293, ['extract', 'yield', 'yield', 'describe', 'use'])
('VBD', 'VBN', 258, ['ended', 'requested', 'revised', 'found', 'Posted'])
('NNP', 'IN', 258, ['though', 'that', 'that', 'that', 'that'])


**Report your findings in the cell below.**  
What kinds of errors did the model make and why do you think it made them? Write a short paragraph (4-5 sentences) in the cell below.

Looking at the printed list, we notice that the most common error is labeling a noun (NN) as a proper noun (NNP). It is interesting noticing that out of the 10 most common errors, 7 involve tagging the word as a plural noun (NNP) instead of something else. The model, thus, learnt in the 'wrong' way how to properly use this tag (maybe due to a particular frequency in the training set).
The model also tags some past participle verbs (VBN) as past tense verbs (VBD). This is most likely due to the fact that in English many verbs have the same form in both past participle and past tense cases. I think the character-level POSTagger won't be able to solve this issue. Overall, the model performes a considerable amount of errors, whose quantity might be decreased with the character-level analysis. 

### Part 2.5 Hyper-parameter Tuning

In order to improve your model performance, try making some modifications on `EMBEDDING_DIM`, `HIDDEN_DIM`, and `LEARNING_RATE`. You will receive 50%/75%/100% credit for this section if your model, after being trained for 10 epochs, is able to achieve 80%/85%/90% accuracy on the validation set.

In [19]:
YOUR_EMBEDDING_DIM = 2
YOUR_HIDDEN_DIM = 6
YOUR_LEARNING_RATE = 0.01

#############################################################################
# TODO: Set three hyper-parameters. Initialize the model, optimizer and the loss function
# Hint, you may want to use reduction='sum' in the CrossEntropyLoss function
#############################################################################
model = BasicPOSTagger(vocab_size    = len(word_to_idx.keys()),
                       embedding_dim = YOUR_EMBEDDING_DIM,
                       hidden_dim    = YOUR_HIDDEN_DIM,
                       tagset_size   = len(tag_to_idx.keys())).to(device)

loss_function = torch.nn.CrossEntropyLoss(reduction='sum').to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=YOUR_LEARNING_RATE)


#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    train(epoch, model, loss_function, optimizer)

Epoch: 1/10	Avg Train Loss: 1.0988	Avg Val Loss: 0.6504	 Val Accuracy: 83
Epoch: 2/10	Avg Train Loss: 0.4905	Avg Val Loss: 0.4792	 Val Accuracy: 88
Epoch: 3/10	Avg Train Loss: 0.3493	Avg Val Loss: 0.4308	 Val Accuracy: 89
Epoch: 4/10	Avg Train Loss: 0.2856	Avg Val Loss: 0.4034	 Val Accuracy: 90
Epoch: 5/10	Avg Train Loss: 0.2505	Avg Val Loss: 0.3939	 Val Accuracy: 90
Epoch: 6/10	Avg Train Loss: 0.2280	Avg Val Loss: 0.3859	 Val Accuracy: 91
Epoch: 7/10	Avg Train Loss: 0.2144	Avg Val Loss: 0.3836	 Val Accuracy: 91
Epoch: 8/10	Avg Train Loss: 0.2017	Avg Val Loss: 0.3824	 Val Accuracy: 91
Epoch: 9/10	Avg Train Loss: 0.1941	Avg Val Loss: 0.3791	 Val Accuracy: 91
Epoch: 10/10	Avg Train Loss: 0.1878	Avg Val Loss: 0.3816	 Val Accuracy: 91


## Part 3 Character-level POS Tagger  [15 points]

Use the character-level information to augment word embeddings. For example, words that end with -ing or -ly give quite a bit of information about their POS tags. To incorporate this information, run a character-level LSTM on every word to create a character-level representation of the word. Take the last hidden state from the character-level LSTM as the representation and concatenate with the word embedding (as in the BasicPOSTagger) to create a new word representation that captures more information.

In [20]:
# Create char to index mapping
char_to_idx = {}
unique_chars = set()
MAX_WORD_LEN = 0

for sent in train_sentences:
    for word in sent:
        for c in word:
            unique_chars.add(c)
        if len(word) > MAX_WORD_LEN:
            MAX_WORD_LEN = len(word)

for c in unique_chars:
    char_to_idx[c] = len(char_to_idx)
char_to_idx[' '] = len(char_to_idx)


## Aside: Padding

For this project, we are not coding in batches (as you can see, each training loop runs on a single sentence per iteration). However, padding is a very important aspect of training, so we describe it in  the section below.

### How to do padding correctly for the characters?


Assume we have got a sentence ["We", "love", "NLP"]. You are supposed to first prepend a certain number of blank characters to each of the words in this sentence.

How to determine the number of blank characters we need? The calculation of MAX_WORD_LEN is here for help (which we already provide in the starter code). For the given sentence, MAX_WORD_LEN equals 4. Therefore we prepend two blank characters to "We", zero blank character to "love", and one blank character to "NLP". So the resultant padded sentence we get should be ["  We", "love", " NLP"].

Then, we feed all characters in ["  We", "love", " NLP"] into a char-embedding layer, and get a tensor of shape (3, 4, char_embedding_dim). To make this tensor's shape proper for the char-level LSTM (nn.LSTM), we need to transpose this tensor, i.e. swap the first and the second dimension. So we get a tensor of shape (4, 3, char_embedding_dim), where 4 corresponds to seq_len and 3 corresponds to batch_size.

The last thing you need to do is to obtain the last hidden state from the char-level LSTM, and concatenate it with the word embedding, so that you can get an augmented representation of that word.

[This](https://raw.githubusercontent.com/chaojiang06/chaojiang06.github.io/master/TA/spring2022_CS4650/char_padding.png) is an illustration for left padding characters.

### Why doing the padding?
Someone may ask why we want to do such a kind of padding, instead of directly passing each of the character sequences of each word one by one through an LSTM, to get the last hidden state. The reason is that if you don't do padding, then that means you can only implement this process using "for loop". For CharPOSTagger, if you implement it using "for loop", the training time would be approximately 150s (GPU) / 250s (CPU) per epoch, while it would be around 30s (GPU) / 150s (CPU) per epoch if you do the padding and feed your data in batches. Therefore, we strongly recommend you learn how to do the padding and transform your data into batches. In fact, those are quite important concepts which you should get yourself familar with, although it might take you some time.

### Why doing left padding?
Our hypothesis is that the suffixes of English words (e.g., -ly, -ing, etc) are more indicative than prefixes for the part-of-speech (POS). Though LSTM is supposed to be able to handle long sequences, it still lose information along the way and the information closer to the last state (which you use as char-level representations) will be retained better. 

### How to understand the dimention change?
Assume we have got a sentence with 3 words ["We", "love", "NLP"], and assume the dimension of character embedding is 2, the dimension of word embedding is 4, the dimension of word-level LSTM's hidden layer is 5, the dimension of character-level LSTM's hidden layer is 6.

In BasicPOSTagger, the dimension change would be (3x1x4) ----word-level LSTM----> (3x1x5) ----linear layer----> (3x1x44).

In CharPOSTagger, after padding, character embedding, and swapping, the dimension change would be (MAX_WORD_LEN, 3, 2) ----character-level LSTM----> (MAX_WORD_LEN, 3, 6) ----Take the last hidden state----> (3, 6) ----concatenate with word embedings----> (3x1x10) ----word-level LSTM----> (3x1x5) ----linear layer----> (3x1x44).

### Part 3.1 Define CharPOSTagger Model

In [21]:
# New Hyperparameters
EMBEDDING_DIM = 4
HIDDEN_DIM = 8
LEARNING_RATE = 0.1
LSTM_LAYERS = 1
DROPOUT = 0
EPOCHS = 10
CHAR_EMBEDDING_DIM = 4
CHAR_HIDDEN_DIM = 4

In [22]:
class CharPOSTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, char_embedding_dim, 
                 char_hidden_dim, char_size, vocab_size, tagset_size):
        super(CharPOSTagger, self).__init__()
        #############################################################################
        # TODO: Define and initialize anything needed for the forward pass.
        # You are required to create a model with:
        # an embedding layer for word: that maps words to their embedding space
        # an embedding layer for character: that maps characters to their embedding space
        # a character-level LSTM layer: that finds the character-level embedding for a word
        # a word-level LSTM layer: that takes the concatenated representation per word (word embedding + char-lstm) as input and outputs hidden states
        # a linear layer: maps from hidden state space to tag space
        #############################################################################
        self.word_emb = nn.Embedding(vocab_size, embedding_dim)
        self.char_emb = nn.Embedding(char_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim, num_layers=LSTM_LAYERS)
        self.lstm = nn.LSTM(embedding_dim + char_hidden_dim, hidden_dim, num_layers=LSTM_LAYERS, bidirectional = True)
        self.linear = nn.Linear(hidden_dim*2, tagset_size)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def forward(self, sentence, chars):
        tag_scores = None
        #############################################################################
        # TODO: Implement the forward pass.
        # Given a tokenized index-mapped sentence and a character sequence as the arguments, 
        # find the corresponding raw scores for tags (without softmax)
        # returns:: tag_scores (Tensor)
        #############################################################################

        word_emb = self.word_emb(sentence)
        char_emb = self.char_emb(chars)

        char_emb = char_emb.transpose(0,1)

        lstm_out_char, (hidden_char, c_char) = self.char_lstm(char_emb)

        concatenated = torch.cat([word_emb, hidden_char[-1]], dim=1).unsqueeze(1)
        lstm_out_word, (hidden_word, c_word) = self.lstm(concatenated)
        tag_scores = self.linear(lstm_out_word)
    
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        return tag_scores



### Part 3.2 Training and Evaluation

In [23]:

def train_char(epoch, model, loss_function, optimizer):
    model.train()
    train_loss = 0
    train_examples = 0
    t = 0
    for sentence, tags in training_data:
        #############################################################################
        # TODO: Implement the training method
        # Hint: you can use the prepare_sequence method for creating index mappings 
        # for sentences. For constructing character input, you may want to left pad
        # each word to MAX_WORD_LEN first, then use prepare_sequence method to create
        # index  mappings. 
        #############################################################################

        #zero out the parameter gradients
        optimizer.zero_grad()

        #prepare input data (sentences, characters, and gold labels)
        prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
        prep_sentence = prep_sentence.to(device)
        prep_tags = prep_tags.to(device)

        MAX_WORD_LEN = max(len(word) for word in sentence)
        padded_words = [word.rjust(MAX_WORD_LEN) for word in sentence]

        chars_seq = []

        for idx, word in enumerate(padded_words):
          prep_chars, _ = prepare_sequence(word, tags, char_to_idx, tag_to_idx)
          prep_chars = prep_chars.to(device)
          chars_seq.append(prep_chars)

        chars_seq = torch.stack(chars_seq, dim=0)
        chars_seq = chars_seq.to(device)
        #do forward pass with current batch of input
        output = model(prep_sentence, chars_seq)

        
        output = output.transpose(0,1).squeeze(dim=0)

        #get loss with model predictions and true labels
        loss = loss_function(output, prep_tags)
        #update model parameters
        loss.backward()
        optimizer.step()
        #increase running total loss and the number of past training samples 
        train_loss += loss.item()
        train_examples += len(tags)
        
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
    
    avg_train_loss = train_loss / train_examples
    avg_val_loss, val_accuracy = evaluate_char(model, loss_function)
        
    print("Epoch: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\t Val Accuracy: {:.0f}".format(epoch, 
                                                                      EPOCHS, 
                                                                      avg_train_loss, 
                                                                      avg_val_loss,
                                                                      val_accuracy))


def evaluate_char(model, loss_function):
    # returns:: avg_val_loss (float)
    # returns:: val_accuracy (float)
    model.eval()
    correct = 0
    val_loss = 0
    val_examples = 0
    with torch.no_grad():
        for sentence, tags in val_data:
            #############################################################################
            # TODO: Implement the evaluate method
            # Find the average validation loss along with the validation accuracy.
            # Hint: To find the accuracy, argmax of tag predictions can be used. 
            #############################################################################

            #prepare input data (sentences, characters, and gold labels)
            prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
            prep_sentence = prep_sentence.to(device)
            prep_tags = prep_tags.to(device)

            MAX_WORD_LEN = max(len(word) for word in sentence)
            padded_words = [word.rjust(MAX_WORD_LEN) for word in sentence]

            
            chars_seq = []
            for idx, word in enumerate(padded_words):
              prep_chars, _ = prepare_sequence(word, tags, char_to_idx, tag_to_idx)
              prep_chars = prep_chars.to(device)
              chars_seq.append(prep_chars)
        
            chars_seq = torch.stack(chars_seq, dim=0)
            chars_seq = chars_seq.to(device)
            #do forward pass with current batch of input
            output = model(prep_sentence, chars_seq)
            output = output.transpose(0,1).squeeze(dim=0)
            #get loss with model predictions and true labels
            loss = loss_function(output, prep_tags)
            
            #get the predicted labels
            predictions = torch.argmax(output, dim=1)
            
            #get number of correct prediction
            correct += (predictions == prep_tags).sum().item()

            #increase running total loss and the number of past valid samples
            val_loss += loss.item()
            val_examples += len(prep_tags)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
    val_accuracy = 100. * correct / val_examples
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, val_accuracy

In [24]:
#############################################################################
# TODO: Initialize the model, optimizer and the loss function
# Hint, you may want to use reduction='sum' in the CrossEntropyLoss function
#############################################################################
model = CharPOSTagger(embedding_dim       = EMBEDDING_DIM, 
                       hidden_dim         = HIDDEN_DIM, 
                       char_embedding_dim = CHAR_EMBEDDING_DIM, 
                       char_hidden_dim    = CHAR_HIDDEN_DIM, 
                       char_size          = len(char_to_idx.keys()), 
                       vocab_size         = len(word_to_idx.keys()), 
                       tagset_size        = len(tag_to_idx.keys())).to(device)

loss_function = torch.nn.CrossEntropyLoss(reduction='sum').to(device) 
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    train_char(epoch, model, loss_function, optimizer)

Epoch: 1/10	Avg Train Loss: 0.8123	Avg Val Loss: 0.5227	 Val Accuracy: 84
Epoch: 2/10	Avg Train Loss: 0.4247	Avg Val Loss: 0.4590	 Val Accuracy: 87
Epoch: 3/10	Avg Train Loss: 0.3356	Avg Val Loss: 0.3710	 Val Accuracy: 90
Epoch: 4/10	Avg Train Loss: 0.2977	Avg Val Loss: 0.3940	 Val Accuracy: 89
Epoch: 5/10	Avg Train Loss: 0.2596	Avg Val Loss: 0.3599	 Val Accuracy: 90
Epoch: 6/10	Avg Train Loss: 0.2651	Avg Val Loss: 0.3835	 Val Accuracy: 90
Epoch: 7/10	Avg Train Loss: 0.2666	Avg Val Loss: 0.5413	 Val Accuracy: 87
Epoch: 8/10	Avg Train Loss: 0.2898	Avg Val Loss: 0.4527	 Val Accuracy: 89
Epoch: 9/10	Avg Train Loss: 0.2519	Avg Val Loss: 0.3931	 Val Accuracy: 90
Epoch: 10/10	Avg Train Loss: 0.2304	Avg Val Loss: 0.3989	 Val Accuracy: 90


**Sanity Check!** Under the default hyperparameter setting, after 5 epochs you should be able to get at least 85% accuracy on the validation set.

### Part 3.3 Error analysis
Write a method to generate predictions for the validation set.
Create lists of words, tags predicted by the model and ground truth tags. 

Then use these lists to carry out error analysis to find the top-10 types of errors made by the model.

This part is very similar to part 1.7. You may want to refer to your implementation there.

In [25]:
#############################################################################
# TODO: Generate predictions for val_data
# Create lists of words, tags predicted by the model and ground truth tags.
# Hint: It should look very similar to the evaluate function.
#############################################################################
def generate_predictions(model, val_data):
    # returns:: word_list (str list)
    # returns:: model_tags (str list)
    # returns:: gt_tags (str list)
    # Your code here

    word_list = []
    gt_tags = []
    model_tags = []
    with torch.no_grad():
        for sentence, tags in val_data:

            prep_sentence, prep_tags = prepare_sequence(sentence, tags, word_to_idx, tag_to_idx)
            prep_sentence = prep_sentence.to(device)
            prep_tags = prep_tags.to(device)

            MAX_WORD_LEN = max(len(word) for word in sentence)
            padded_words = [word.rjust(MAX_WORD_LEN) for word in sentence]

            
            chars_seq = []
            for idx, word in enumerate(padded_words):
              prep_chars, _ = prepare_sequence(word, tags, char_to_idx, tag_to_idx)
              prep_chars = prep_chars.to(device)
              chars_seq.append(prep_chars)
        
            chars_seq = torch.stack(chars_seq, dim=0)
            chars_seq = chars_seq.to(device)

            output = model(prep_sentence, chars_seq)
            output = output.transpose(0,1).squeeze(dim=0)

            predictions = torch.argmax(output, dim=1)

            for word, tag, pred_tag in zip(sentence, prep_tags, predictions):
              word_list.append(word)
              gt_tags.append(idx_to_tag[tag.item()])
              model_tags.append(idx_to_tag[pred_tag.item()])


    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################


    return word_list, model_tags, gt_tags

#############################################################################
# TODO: Carry out error analysis
# From those lists collected from the above method, find the 
# top-10 tuples of (model_tag, ground_truth_tag, frequency, example words)
# sorted by frequency
#############################################################################
def error_analysis(word_list, model_tags, gt_tags):
    # returns: errors (list of tuples)
    # Your code here

    frequencies = {}
    example_words = {}
    for word, model_tag, gt_tag in zip(word_list, model_tags, gt_tags):
      if model_tag != gt_tag: 
        if (model_tag, gt_tag) in frequencies:
          frequencies[(model_tag, gt_tag)] += 1
          example_words[(model_tag, gt_tag)].append(word)
        else:
          frequencies[(model_tag, gt_tag)] = 1
          example_words[(model_tag, gt_tag)] = [word]
    
    errors = []
    sorted_frequencies = dict(sorted(frequencies.items(), key=lambda x: x[1], reverse=True))
    for error in sorted_frequencies:
      e = (error[0], error[1], frequencies[error], example_words[error][:5])
      errors.append(e)
    
    #############################################################################
    #                             END OF YOUR CODE                              #
    #############################################################################

    return errors

word_list, model_tags, gt_tags = generate_predictions(model, val_data)
errors = error_analysis(word_list, model_tags, gt_tags)

for i in errors[:10]:
  print(i)

('NN', 'NNP', 315, ['Bateman', 'Bryan', 'Ivy', 'Wall', 'Wheeling-Pittsburgh'])
('NN', 'JJ', 282, ['gullible', 'greedy', 'Callable', 'available', 'capitalist'])
('NNP', 'JJ', 226, ['slick-talking', 'snake-oil', 'due', 'due', 'lengthy'])
('VBN', 'VBD', 209, ['made', 'permitted', 'conceded', 'continued', 'topped'])
('NNP', 'NN', 178, ['hotdog', 'depressant', 'Market', 'tandem', 'duck'])
('VBP', 'VB', 119, ['have', 'have', 'have', 'have', 'have'])
('NN', 'VB', 108, ['yield', 'describe', 'list', 'EXAMINE', 'boost'])
('VBD', 'VBN', 99, ['ended', 'formed', 'held', 'held', 'approved'])
('VBZ', 'NNS', 96, ['prices', 'eyebrows', 'switches', 'Thousands', 'prices'])
('JJ', 'NN', 93, ['many', 'breakfast', 'wood', 'net', 'defamation'])


**Report your findings in the cell below.**  
What kinds of errors does the character-level model make as compared to the original model, and why do you think it made them? 

For the character-level POS tagging, the most common error is assigning a noun (NN) tag instead of the proper noun tag (NNP). In this model, thanks to the character-level information, the model might find it easier to match words that the previous model had difficulties with. As predicted, the character-level POSTagger still tags some past tense verbs (VBD) as past participle verbs (VBN) and vice-versa (error 4 and 8). This comes from the fact that due to the same exact forms of these words in English, the model is not able to improve on these mistakes, despite the character-level analysis. Also, we can notice that the model tags some nouns (NN) as proper nouns (NNP), like 'hotdog' or 'Market'. This might be due to either a low frequency of the words or the capitalized first letter, that might refer to the word being at the beginning on the sentence. With the character-level analysis, the amount of error has decreased, with respect to the word-level analysis of the previous tagger. 

### Part 4: Submit Your Homework
This is the end. Congratulations!  

Now, follow the steps below to submit your homework in [Gradescope](https://www.gradescope.com/courses/345683):

1. Rename this ipynb file to 'CS4650_p2_GTusername.ipynb'. We recommend ensuring you have removed any extraneous cells & print statements, clearing all outputs, and using the Runtime --> Run all tool to make sure all output is update to date. Additionally, leaving comments in your code to help us understand your operations will assist the teaching staff in grading. It is not a requirement, but is recommended. 
2. Click on the menu 'File' --> 'Download' --> 'Download .py'.
3. Click on the menu 'File' --> 'Download' --> 'Download .ipynb'.
4. Download the notebook as a .pdf document. Make sure the output from your training loops are captured so we can see how the loss and accuracy changes while training.
5. Upload all 3 files to GradeScope.

